# Création du custom scaler

Donc je veux créer un min_max_scaler qui scale toujours de la même manière.


In [7]:
import numpy as np
import pandas as pd
import data_processing as dp

In [3]:
data_clean = pd.read_csv('../data/data_clean3.csv', sep=';', parse_dates=['Datetime']).set_index('Datetime')
prodSol = pd.read_csv('../data/prod/prodSol.csv', sep=';', parse_dates=['Datetime']).set_index('Datetime')
prodEol = pd.read_csv('../data/prod/prodEol.csv', sep=';', parse_dates=['Datetime']).set_index('Datetime')
prodRiv = pd.read_csv('../data/prod/prodRiv.csv', sep=';', parse_dates=['Datetime']).set_index('Datetime')

In [4]:
pv=122
onshore=80
offshore=13

In [8]:
data = pd.merge(data_clean,(prodSol * pv).merge((prodEol * (offshore + onshore)).merge(prodRiv, on='Datetime'),
                                                  on='Datetime'), on='Datetime')
data['Production Residuelle (MW)'] = dp.prodRes(Conso=data['Consommation (MW)'],
                                                             prodEol=data['Eolien (TW)'],
                                                             prodSol=data['Solaire (TW)'],
                                                             prodRiv=data['Hydraulique (MW)'],
                                                             pv=1, onshore=1, offshore=1)  # prod data already processed
data['Annee'] = data.index.year
data['Mois'] = data.index.month
data['Jour'] = data.index.day
data['Heure'] = data.index.hour
data['Minute'] = data.index.minute

In [54]:
data

,Pression au niveau mer,Variation de pression en 3 heures,Température,Humidité,Consommation (MW),Holiday,A,B,C,Confinement,...,Prix Elec 1kWh,Solaire (TW),Eolien (TW),Hydraulique (MW),Production Residuelle (MW),Annee,Mois,Jour,Heure,Minute
Datetime,,,,,,,,,,,,,,,,,,,,,
2012-01-01 01:00:00,101912.1325,-26.204878,284.441001,90.875456,56231.0,True,True,True,True,0,...,69.37,0.000000,63.768199,7598.0,78903.398418,2012,1,1,1,0
2012-01-01 01:15:00,101909.5278,-26.399241,284.418138,91.010688,56153.0,True,True,True,True,0,...,69.37,0.000000,63.678029,7448.5,78651.558488,2012,1,1,1,15
2012-01-01 01:30:00,101906.9230,-26.593603,284.395274,91.145920,56075.0,True,True,True,True,0,...,69.37,0.000000,63.587859,7299.0,78399.718558,2012,1,1,1,30
2012-01-01 01:45:00,101904.3182,-26.787965,284.372410,91.281152,55803.5,True,True,True,True,0,...,69.37,0.000000,63.669012,7229.0,78763.524495,2012,1,1,1,45
2012-01-01 02:00:00,101901.7134,-26.982327,284.349546,91.416384,55532.0,True,True,True,True,0,...,69.37,0.000000,63.750165,7159.0,79127.330432,2012,1,1,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-10 13:45:00,101690.4182,-5.555157,288.633609,67.142344,48632.0,False,False,False,False,0,...,104.08,85.847085,35.917932,6661.0,115711.948953,2023,5,10,13,45
2023-05-10 14:00:00,101688.5445,-11.812752,288.791610,66.369890,48073.0,False,False,False,False,0,...,104.08,83.554008,37.196585,6695.0,116569.176900,2023,5,10,14,0
2023-05-10 14:15:00,101683.9215,-15.072847,288.828798,66.207288,48427.0,False,False,False,False,0,...,104.08,86.745207,39.156329,6784.0,123414.864377,2023,5,10,14,15


In [19]:
mean = np.mean(data.drop(columns=['Annee']), axis=0)

In [18]:
std = np.std(data.drop(columns=['Annee']), axis=0)

In [31]:
data_centree_reduite = (data.drop(columns=['Annee']) - mean)/(std)
data_centree_reduite

,Pression au niveau mer,Variation de pression en 3 heures,Température,Humidité,Consommation (MW),Holiday,A,B,C,Confinement,...,week-end,Prix Elec 1kWh,Solaire (TW),Eolien (TW),Hydraulique (MW),Production Residuelle (MW),Mois,Jour,Heure,Minute
Datetime,,,,,,,,,,,,,,,,,,,,,
2012-01-01 01:00:00,0.221722,-0.293664,-0.150397,1.242184,0.209950,5.684344,1.40552,1.404764,1.404764,-0.182028,...,1.571286,-0.276826,-0.700552,1.484390,0.196859,0.949049,-1.555056,-1.672149,-1.511075,-1.341641
2012-01-01 01:15:00,0.219276,-0.295804,-0.153679,1.252538,0.203411,5.684344,1.40552,1.404764,1.404764,-0.182028,...,1.571286,-0.276826,-0.700552,1.480398,0.143013,0.943785,-1.555056,-1.672149,-1.511075,-0.447214
2012-01-01 01:30:00,0.216829,-0.297944,-0.156962,1.262893,0.196872,5.684344,1.40552,1.404764,1.404764,-0.182028,...,1.571286,-0.276826,-0.700552,1.476407,0.089167,0.938521,-1.555056,-1.672149,-1.511075,0.447214
2012-01-01 01:45:00,0.214382,-0.300084,-0.160244,1.273247,0.174110,5.684344,1.40552,1.404764,1.404764,-0.182028,...,1.571286,-0.276826,-0.700552,1.479999,0.063954,0.946125,-1.555056,-1.672149,-1.511075,1.341641
2012-01-01 02:00:00,0.211936,-0.302224,-0.163526,1.283602,0.151349,5.684344,1.40552,1.404764,1.404764,-0.182028,...,1.571286,-0.276826,-0.700552,1.483591,0.038742,0.953730,-1.555056,-1.672149,-1.366736,-1.341641
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-10 13:45:00,0.013466,-0.066307,0.451503,-0.575040,-0.427122,-0.175922,-0.71148,-0.711863,-0.711863,-0.182028,...,-0.636421,1.407554,2.350792,0.251599,-0.140626,1.718481,-0.401137,-0.651456,0.220993,1.341641
2023-05-10 14:00:00,0.011706,-0.135204,0.474186,-0.634186,-0.473987,-0.175922,-0.71148,-0.711863,-0.711863,-0.182028,...,-0.636421,1.407554,2.269287,0.308199,-0.128380,1.736400,-0.401137,-0.651456,0.365332,-1.341641
2023-05-10 14:15:00,0.007364,-0.171098,0.479525,-0.646636,-0.444309,-0.175922,-0.71148,-0.711863,-0.711863,-0.182028,...,-0.636421,1.407554,2.382715,0.394947,-0.096324,1.879500,-0.401137,-0.651456,0.365332,-0.447214


In [32]:
np.std(data_centree_reduite['Holiday'])

1.0

In [33]:
data_scaled = np.tanh(data_centree_reduite)

In [34]:
data_scaled

,Pression au niveau mer,Variation de pression en 3 heures,Température,Humidité,Consommation (MW),Holiday,A,B,C,Confinement,...,week-end,Prix Elec 1kWh,Solaire (TW),Eolien (TW),Hydraulique (MW),Production Residuelle (MW),Mois,Jour,Heure,Minute
Datetime,,,,,,,,,,,,,,,,,,,,,
2012-01-01 01:00:00,0.218159,-0.285504,-0.149273,0.846077,0.206919,0.999977,0.886539,0.886377,0.886377,-0.180044,...,0.917230,-0.269965,-0.604718,0.902287,0.194355,0.739352,-0.914616,-0.931835,-0.907130,-0.872066
2012-01-01 01:15:00,0.215828,-0.287468,-0.152481,0.848994,0.200651,0.999977,0.886539,0.886377,0.886377,-0.180044,...,0.917230,-0.269965,-0.604718,0.901543,0.142046,0.736956,-0.914616,-0.931835,-0.907130,-0.419606
2012-01-01 01:30:00,0.213494,-0.289430,-0.155685,0.851860,0.194367,0.999977,0.886539,0.886377,0.886377,-0.180044,...,0.917230,-0.269965,-0.604718,0.900793,0.088931,0.734542,-0.914616,-0.931835,-0.907130,0.419606
2012-01-01 01:45:00,0.211157,-0.291389,-0.158886,0.854675,0.172372,0.999977,0.886539,0.886377,0.886377,-0.180044,...,0.917230,-0.269965,-0.604718,0.901468,0.063867,0.738024,-0.914616,-0.931835,-0.907130,0.872066
2012-01-01 02:00:00,0.208818,-0.293346,-0.162084,0.857442,0.150204,0.999977,0.886539,0.886377,0.886377,-0.180044,...,0.917230,-0.269965,-0.604718,0.902139,0.038723,0.741467,-0.914616,-0.931835,-0.877946,-0.872066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-10 13:45:00,0.013466,-0.066210,0.423134,-0.519051,-0.402914,-0.174129,-0.611604,-0.611844,-0.611844,-0.180044,...,-0.562458,0.886974,0.982002,0.246421,-0.139706,0.937680,-0.380922,-0.572649,0.217465,0.872066
2023-05-10 14:00:00,0.011706,-0.134386,0.441576,-0.560928,-0.441415,-0.174129,-0.611604,-0.611844,-0.611844,-0.180044,...,-0.562458,0.886974,0.978849,0.298797,-0.127679,0.939808,-0.380922,-0.572649,0.349902,-0.872066
2023-05-10 14:15:00,0.007364,-0.169448,0.445863,-0.569401,-0.417210,-0.174129,-0.611604,-0.611844,-0.611844,-0.180044,...,-0.562458,0.886974,0.983105,0.375617,-0.096027,0.954448,-0.380922,-0.572649,0.349902,-0.419606


In [35]:
data_scaled.to_numpy()

array([[ 0.21815899, -0.28550363, -0.14927325, ..., -0.93183519,
        -0.90712965, -0.87206578],
       [ 0.2158276 , -0.28746796, -0.1524809 , ..., -0.93183519,
        -0.90712965, -0.41960589],
       [ 0.21349366, -0.28942987, -0.15568533, ..., -0.93183519,
        -0.90712965,  0.41960589],
       ...,
       [ 0.00736384, -0.169448  ,  0.44586331, ..., -0.57264926,
         0.34990215, -0.41960589],
       [ 0.00302168, -0.20408607,  0.45013058, ..., -0.57264926,
         0.34990215,  0.41960589],
       [-0.00132068, -0.23822057,  0.45437739, ..., -0.57264926,
         0.34990215,  0.87206578]])

In [36]:
data_unscaled = np.arctanh(data_scaled)*std + mean
data_unscaled

c:\users\doria\appdata\local\programs\python\python38\lib\site-packages\pandas\core\internals\blocks.py:329: RuntimeWarning: divide by zero encountered in arctanh
  result = func(self.values, **kwargs)


,Pression au niveau mer,Variation de pression en 3 heures,Température,Humidité,Consommation (MW),Holiday,A,B,C,Confinement,...,week-end,Prix Elec 1kWh,Solaire (TW),Eolien (TW),Hydraulique (MW),Production Residuelle (MW),Mois,Jour,Heure,Minute
Datetime,,,,,,,,,,,,,,,,,,,,,
2012-01-01 01:00:00,101912.1325,-26.204878,284.441001,90.875456,56231.0,1.000000e+00,1.0,1.0,1.000000e+00,0.0,...,1.0,69.37,0.000000,63.768199,7598.0,78903.398418,1.0,1.0,1.0,0.0
2012-01-01 01:15:00,101909.5278,-26.399241,284.418138,91.010688,56153.0,1.000000e+00,1.0,1.0,1.000000e+00,0.0,...,1.0,69.37,0.000000,63.678029,7448.5,78651.558488,1.0,1.0,1.0,15.0
2012-01-01 01:30:00,101906.9230,-26.593603,284.395274,91.145920,56075.0,1.000000e+00,1.0,1.0,1.000000e+00,0.0,...,1.0,69.37,0.000000,63.587859,7299.0,78399.718558,1.0,1.0,1.0,30.0
2012-01-01 01:45:00,101904.3182,-26.787965,284.372410,91.281152,55803.5,1.000000e+00,1.0,1.0,1.000000e+00,0.0,...,1.0,69.37,0.000000,63.669012,7229.0,78763.524495,1.0,1.0,1.0,45.0
2012-01-01 02:00:00,101901.7134,-26.982327,284.349546,91.416384,55532.0,1.000000e+00,1.0,1.0,1.000000e+00,0.0,...,1.0,69.37,0.000000,63.750165,7159.0,79127.330432,1.0,1.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-10 13:45:00,101690.4182,-5.555157,288.633609,67.142344,48632.0,-3.469447e-18,0.0,0.0,5.551115e-17,0.0,...,0.0,104.08,85.847085,35.917932,6661.0,115711.948953,5.0,10.0,13.0,45.0
2023-05-10 14:00:00,101688.5445,-11.812752,288.791610,66.369890,48073.0,-3.469447e-18,0.0,0.0,5.551115e-17,0.0,...,0.0,104.08,83.554008,37.196585,6695.0,116569.176900,5.0,10.0,14.0,0.0
2023-05-10 14:15:00,101683.9215,-15.072847,288.828798,66.207288,48427.0,-3.469447e-18,0.0,0.0,5.551115e-17,0.0,...,0.0,104.08,86.745207,39.156329,6784.0,123414.864377,5.0,10.0,14.0,15.0


In [41]:
export_mean_std = pd.concat((mean,std), axis=1)
export_mean_std

,0,1
Pression au niveau mer,101676.081632,1064.624005
Variation de pression en 3 heures,0.467160,90.825030
Température,285.488609,6.965617
Humidité,74.652418,13.060096
Consommation (MW),53726.713835,11927.994967
Holiday,0.030019,0.170641
A,0.336079,0.472366
B,0.336320,0.472450
C,0.336320,0.472450
Confinement,0.032071,0.176190


In [46]:
data_scaled1 = np.tanh(
    (data.drop(columns=['Annee']) - export_mean_std[0])/export_mean_std[1]
)

In [47]:
data_scaled1

,Pression au niveau mer,Variation de pression en 3 heures,Température,Humidité,Consommation (MW),Holiday,A,B,C,Confinement,...,week-end,Prix Elec 1kWh,Solaire (TW),Eolien (TW),Hydraulique (MW),Production Residuelle (MW),Mois,Jour,Heure,Minute
Datetime,,,,,,,,,,,,,,,,,,,,,
2012-01-01 01:00:00,0.218159,-0.285504,-0.149273,0.846077,0.206919,0.999977,0.886539,0.886377,0.886377,-0.180044,...,0.917230,-0.269965,-0.604718,0.902287,0.194355,0.739352,-0.914616,-0.931835,-0.907130,-0.872066
2012-01-01 01:15:00,0.215828,-0.287468,-0.152481,0.848994,0.200651,0.999977,0.886539,0.886377,0.886377,-0.180044,...,0.917230,-0.269965,-0.604718,0.901543,0.142046,0.736956,-0.914616,-0.931835,-0.907130,-0.419606
2012-01-01 01:30:00,0.213494,-0.289430,-0.155685,0.851860,0.194367,0.999977,0.886539,0.886377,0.886377,-0.180044,...,0.917230,-0.269965,-0.604718,0.900793,0.088931,0.734542,-0.914616,-0.931835,-0.907130,0.419606
2012-01-01 01:45:00,0.211157,-0.291389,-0.158886,0.854675,0.172372,0.999977,0.886539,0.886377,0.886377,-0.180044,...,0.917230,-0.269965,-0.604718,0.901468,0.063867,0.738024,-0.914616,-0.931835,-0.907130,0.872066
2012-01-01 02:00:00,0.208818,-0.293346,-0.162084,0.857442,0.150204,0.999977,0.886539,0.886377,0.886377,-0.180044,...,0.917230,-0.269965,-0.604718,0.902139,0.038723,0.741467,-0.914616,-0.931835,-0.877946,-0.872066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-10 13:45:00,0.013466,-0.066210,0.423134,-0.519051,-0.402914,-0.174129,-0.611604,-0.611844,-0.611844,-0.180044,...,-0.562458,0.886974,0.982002,0.246421,-0.139706,0.937680,-0.380922,-0.572649,0.217465,0.872066
2023-05-10 14:00:00,0.011706,-0.134386,0.441576,-0.560928,-0.441415,-0.174129,-0.611604,-0.611844,-0.611844,-0.180044,...,-0.562458,0.886974,0.978849,0.298797,-0.127679,0.939808,-0.380922,-0.572649,0.349902,-0.872066
2023-05-10 14:15:00,0.007364,-0.169448,0.445863,-0.569401,-0.417210,-0.174129,-0.611604,-0.611844,-0.611844,-0.180044,...,-0.562458,0.886974,0.983105,0.375617,-0.096027,0.954448,-0.380922,-0.572649,0.349902,-0.419606


In [48]:
data_unscaled1 = np.arctanh(data_scaled1)*std + mean
data_unscaled1

c:\users\doria\appdata\local\programs\python\python38\lib\site-packages\pandas\core\internals\blocks.py:329: RuntimeWarning: divide by zero encountered in arctanh
  result = func(self.values, **kwargs)


,Pression au niveau mer,Variation de pression en 3 heures,Température,Humidité,Consommation (MW),Holiday,A,B,C,Confinement,...,week-end,Prix Elec 1kWh,Solaire (TW),Eolien (TW),Hydraulique (MW),Production Residuelle (MW),Mois,Jour,Heure,Minute
Datetime,,,,,,,,,,,,,,,,,,,,,
2012-01-01 01:00:00,101912.1325,-26.204878,284.441001,90.875456,56231.0,1.000000e+00,1.0,1.0,1.000000e+00,0.0,...,1.0,69.37,0.000000,63.768199,7598.0,78903.398418,1.0,1.0,1.0,0.0
2012-01-01 01:15:00,101909.5278,-26.399241,284.418138,91.010688,56153.0,1.000000e+00,1.0,1.0,1.000000e+00,0.0,...,1.0,69.37,0.000000,63.678029,7448.5,78651.558488,1.0,1.0,1.0,15.0
2012-01-01 01:30:00,101906.9230,-26.593603,284.395274,91.145920,56075.0,1.000000e+00,1.0,1.0,1.000000e+00,0.0,...,1.0,69.37,0.000000,63.587859,7299.0,78399.718558,1.0,1.0,1.0,30.0
2012-01-01 01:45:00,101904.3182,-26.787965,284.372410,91.281152,55803.5,1.000000e+00,1.0,1.0,1.000000e+00,0.0,...,1.0,69.37,0.000000,63.669012,7229.0,78763.524495,1.0,1.0,1.0,45.0
2012-01-01 02:00:00,101901.7134,-26.982327,284.349546,91.416384,55532.0,1.000000e+00,1.0,1.0,1.000000e+00,0.0,...,1.0,69.37,0.000000,63.750165,7159.0,79127.330432,1.0,1.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-10 13:45:00,101690.4182,-5.555157,288.633609,67.142344,48632.0,-3.469447e-18,0.0,0.0,5.551115e-17,0.0,...,0.0,104.08,85.847085,35.917932,6661.0,115711.948953,5.0,10.0,13.0,45.0
2023-05-10 14:00:00,101688.5445,-11.812752,288.791610,66.369890,48073.0,-3.469447e-18,0.0,0.0,5.551115e-17,0.0,...,0.0,104.08,83.554008,37.196585,6695.0,116569.176900,5.0,10.0,14.0,0.0
2023-05-10 14:15:00,101683.9215,-15.072847,288.828798,66.207288,48427.0,-3.469447e-18,0.0,0.0,5.551115e-17,0.0,...,0.0,104.08,86.745207,39.156329,6784.0,123414.864377,5.0,10.0,14.0,15.0


In [51]:
np.double(1.)

1.0

In [52]:
np.float64(1.)

1.0

In [53]:
export_mean_std.to_csv('../data/mean_std.csv', sep=';')

In [59]:
print(np.finfo(np.float128))

AttributeError: module 'numpy' has no attribute 'float128'

In [69]:
mean_std = pd.read_csv('../data/mean_std.csv', sep=';').set_index('Unnamed: 0')

In [72]:
mean_std

,0,1
Unnamed: 0,,
Pression au niveau mer,101676.081632,1064.624005
Variation de pression en 3 heures,0.467160,90.825030
Température,285.488609,6.965617
Humidité,74.652418,13.060096
Consommation (MW),53726.713835,11927.994967
Holiday,0.030019,0.170641
A,0.336079,0.472366
B,0.336320,0.472450
C,0.336320,0.472450


In [73]:
mean_std['0']

Unnamed: 0
Pression au niveau mer               101676.081632
Variation de pression en 3 heures         0.467160
Température                             285.488609
Humidité                                 74.652418
Consommation (MW)                     53726.713835
Holiday                                   0.030019
A                                         0.336079
B                                         0.336320
C                                         0.336320
Confinement                               0.032071
Prévision J (MW)                      53411.036919
Prévision J-1 (MW)                    53488.219292
week-end                                  0.288273
Prix Elec 1kWh                           75.074551
Solaire (TW)                             19.709471
Eolien (TW)                              30.233996
Hydraulique (MW)                       7051.435582
Production Residuelle (MW)            33502.184454
Mois                                      6.390523
Jour                

In [74]:
data_scaled2 = np.tanh(
    (data.drop(columns=['Annee']) - mean_std['0'])/mean_std['1']
)

In [75]:
data_scaled2

,Pression au niveau mer,Variation de pression en 3 heures,Température,Humidité,Consommation (MW),Holiday,A,B,C,Confinement,...,week-end,Prix Elec 1kWh,Solaire (TW),Eolien (TW),Hydraulique (MW),Production Residuelle (MW),Mois,Jour,Heure,Minute
Datetime,,,,,,,,,,,,,,,,,,,,,
2012-01-01 01:00:00,0.218159,-0.285504,-0.149273,0.846077,0.206919,0.999977,0.886539,0.886377,0.886377,-0.180044,...,0.917230,-0.269965,-0.604718,0.902287,0.194355,0.739352,-0.914616,-0.931835,-0.907130,-0.872066
2012-01-01 01:15:00,0.215828,-0.287468,-0.152481,0.848994,0.200651,0.999977,0.886539,0.886377,0.886377,-0.180044,...,0.917230,-0.269965,-0.604718,0.901543,0.142046,0.736956,-0.914616,-0.931835,-0.907130,-0.419606
2012-01-01 01:30:00,0.213494,-0.289430,-0.155685,0.851860,0.194367,0.999977,0.886539,0.886377,0.886377,-0.180044,...,0.917230,-0.269965,-0.604718,0.900793,0.088931,0.734542,-0.914616,-0.931835,-0.907130,0.419606
2012-01-01 01:45:00,0.211157,-0.291389,-0.158886,0.854675,0.172372,0.999977,0.886539,0.886377,0.886377,-0.180044,...,0.917230,-0.269965,-0.604718,0.901468,0.063867,0.738024,-0.914616,-0.931835,-0.907130,0.872066
2012-01-01 02:00:00,0.208818,-0.293346,-0.162084,0.857442,0.150204,0.999977,0.886539,0.886377,0.886377,-0.180044,...,0.917230,-0.269965,-0.604718,0.902139,0.038723,0.741467,-0.914616,-0.931835,-0.877946,-0.872066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-10 13:45:00,0.013466,-0.066210,0.423134,-0.519051,-0.402914,-0.174129,-0.611604,-0.611844,-0.611844,-0.180044,...,-0.562458,0.886974,0.982002,0.246421,-0.139706,0.937680,-0.380922,-0.572649,0.217465,0.872066
2023-05-10 14:00:00,0.011706,-0.134386,0.441576,-0.560928,-0.441415,-0.174129,-0.611604,-0.611844,-0.611844,-0.180044,...,-0.562458,0.886974,0.978849,0.298797,-0.127679,0.939808,-0.380922,-0.572649,0.349902,-0.872066
2023-05-10 14:15:00,0.007364,-0.169448,0.445863,-0.569401,-0.417210,-0.174129,-0.611604,-0.611844,-0.611844,-0.180044,...,-0.562458,0.886974,0.983105,0.375617,-0.096027,0.954448,-0.380922,-0.572649,0.349902,-0.419606


In [77]:
df_2022 = data_clean[data_clean.index.year != 2022]
df_2022

,Pression au niveau mer,Variation de pression en 3 heures,Température,Humidité,Consommation (MW),Holiday,A,B,C,Confinement,Prévision J (MW),Prévision J-1 (MW),week-end,Prix Elec 1kWh
Datetime,,,,,,,,,,,,,,
2012-01-01 01:00:00,101912.1325,-26.204878,284.441001,90.875456,56231.0,True,True,True,True,0,55100,55200,1,69.37
2012-01-01 01:15:00,101909.5278,-26.399241,284.418138,91.010688,56153.0,True,True,True,True,0,55250,55600,1,69.37
2012-01-01 01:30:00,101906.9230,-26.593603,284.395274,91.145920,56075.0,True,True,True,True,0,55400,56000,1,69.37
2012-01-01 01:45:00,101904.3182,-26.787965,284.372410,91.281152,55803.5,True,True,True,True,0,55200,56000,1,69.37
2012-01-01 02:00:00,101901.7134,-26.982327,284.349546,91.416384,55532.0,True,True,True,True,0,55000,56000,1,69.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-10 13:45:00,101690.4182,-5.555157,288.633609,67.142344,48632.0,False,False,False,False,0,48350,49900,0,104.08
2023-05-10 14:00:00,101688.5445,-11.812752,288.791610,66.369890,48073.0,False,False,False,False,0,48300,49700,0,104.08
2023-05-10 14:15:00,101683.9215,-15.072847,288.828798,66.207288,48427.0,False,False,False,False,0,47850,49600,0,104.08


In [80]:
prodSol[prodSol.index.year != 2022]

,Solaire (TW)
Datetime,
2012-01-01 00:00:00,0.000000
2012-01-01 00:15:00,0.000000
2012-01-01 00:30:00,0.000000
2012-01-01 00:45:00,0.000000
2012-01-01 01:00:00,0.000000
...,...
2023-05-12 22:45:00,0.493231
2023-05-12 23:00:00,0.493231
2023-05-12 23:15:00,0.493231


In [85]:
flux = [
    [-20,-5,-7,-9],
    [-27,-15,-12,-7]
]

prodRes = [-44,-69]

prodRes - np.sum(flux, axis=1)

array([-3, -8])

In [86]:
np.sum(prodRes - np.sum(flux, axis=1)) * -0.25

2.75